In [22]:
# Generate the oring of the one hot feats for each word in the title
import re
import pickle
from collections import OrderedDict
import numpy as np

In [11]:
class Page:
    def __init__(self, text, links):
        self.text = text
        self.links = links  # out-links
        self.in_links = []
        self.indx = None  # Relative to the ordered dict below
        self.feats = None

In [12]:
# Load the wiki-dict i want
with open('data/animals-D3-small-30K-nodes40-edges202-max10-minout2-minin3.pkl', 'rb') as f:
    pages = pickle.load(f)
pages = OrderedDict(pages)

In [29]:
replace_chars = lambda title: re.sub(r"[()]", '', title)

In [32]:
def create_dict(titles):
    # Split each title into its words and ignore ()
    words = set()
    for title in titles:
        title = replace_chars(title)
        title_words = title.split()
        words.update(title_words)
    return words

In [33]:
titles = list(pages.keys())
words_set = create_dict(titles)

In [54]:
def one_hot_encode(words_set):
    num_words = len(words_set)
    print('Dict length: {}'.format(num_words))
    dic = {}
    for i, word in enumerate(words_set):
        encoding = np.zeros(shape=(num_words,), dtype=np.int)
        encoding[i] = 1
        dic[word] = encoding
    return dic

In [55]:
encoding_dic = one_hot_encode(words_set)

Dict length: 42


In [62]:
def add_node_feats(pages, dic):
    for title, obj in list(pages.items()):
        title_encode = np.zeros(shape=(len(dic,)), dtype=np.int)  # Init encode is all zeros since im oring
        title_ = replace_chars(title)
        title_words = title_.split()
        for word in title_words:
            word_encoding = dic[word]
            title_encode = np.logical_or(title_encode, word_encoding, dtype=np.int).astype(np.int)
        obj.feats = title_encode

In [63]:
add_node_feats(pages, encoding_dic)

In [66]:
with open('data/animals-D3-small-30K-nodes40-edges202-max10-minout2-minin3_w_features.pkl', 'wb') as f:
    pickle.dump(pages, f)